<a href="https://colab.research.google.com/github/spoupry/AQdata/blob/main/AggregatorVF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Programme de l'aggrégateur pour traitement/visualisation des données
New features:
1. construction des indices de défaillance poour chaque fenêtre rpoulante
2. Stockage des Ids dans détections

Features:

1.  Récupère les données brutes d'une station de mesure en csv
2. Charges dans un dataframe pour effectuer des traitements sur chaque paramètre
3. Détecter les fautes capteurs de la station
4. Fourni une synthèse de chaque paramètre
5. affiche une vue capteur pour fixer les seuils de détections
6. affiche une vue synthèse d'un polluant en particulier avec données météorologiques locales
7. affiche une vue comparaison avec ajout données extérieures pour comparaison

# Modules


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.nonparametric.kernel_regression import KernelReg
from numpy import linspace
from datetime import datetime, timedelta
from scipy import stats

# Importation données locales

In [ ]:
# Import data from GitHub
url1 = 'https://github.com/spoupry/AQdata/raw/main/data_ARG_dec21-june21_5_parameters.csv'
# Read and use csv file Local measures from Argeles-gazost
dfin = pd.read_csv(url1, delimiter=",")
dfin['time'] = pd.to_datetime(dfin['time'])
# Setting the timestamp as index of the dataframe. THis helps to iterate through the data based on time
dfin.set_index('time', inplace=True)
print(dfin.head(1))
print(dfin.tail(1))
dfin.dtypes
# data ready to use !

                    Id_Smart_Capteur  Humidite  Pression  PM10  PM2.5  \
time                                                                    
2021-12-08 15:13:25         Arg12b1f  37.36719  954.4344   6.2    2.0   

                     Temperature  
time                              
2021-12-08 15:13:25        19.13  
                    Id_Smart_Capteur  Humidite  Pression  PM10  PM2.5  \
time                                                                    
2022-06-18 00:41:20         Arg12b2f  36.96875  966.4397   0.0    0.0   

                     Temperature  
time                              
2022-06-18 00:41:20        34.37  


Id_Smart_Capteur     object
Humidite            float64
Pression            float64
PM10                float64
PM2.5               float64
Temperature         float64
dtype: object

In [ ]:
#@title Select data range (2021-12-08 to 2022-05-22). { run: "auto" }
#@title Click `Show code` in the code cell. { display-mode: "form" }
#@markdown ---
#@markdown ### Enter start and end dates here:
date_debut = '2022-02-08' #@param {type:"date"}
suggestion_fin =  str(datetime.strptime(date_debut,'%Y-%m-%d').date() + timedelta(days = 15))
print("suggestion de date de fin:",suggestion_fin, " (15 jours)")
date_fin = "2022-02-09" #@param {type:"date"}
#@markdown ---
print('date début:', date_debut, 'date fin:',date_fin)
data = dfin.loc[date_debut : date_fin]
print(data[:1])

suggestion de date de fin: 2022-02-23  (15 jours)
date début: 2022-02-08 date fin: 2022-02-09
                    Id_Smart_Capteur  Humidite  Pression  PM10  PM2.5  \
time                                                                    
2022-02-08 00:03:30         Arg12b2f  43.10352  979.5614   8.4    4.3   

                     Temperature  
time                              
2022-02-08 00:03:30        18.08  


# Fonctions annexes

In [ ]:
# Calculate the median of a list
# Input : time-serie
# Output the median as a float or None if list =[]
def calculate_median(l):
    l = sorted(l)
    l_len = len(l)
    if l_len < 1:
        return None
    if l_len % 2 == 0 :
        return ( l[(l_len)//2] + l[(l_len//2)-1] ) / 2.0
    else:
        return l[(l_len-1)//2]

# Renvoie l'élément manquant par comparaison des deux listes
# IN: liste de référence, liste à contrôler
# Out: une liste des élements manquants
def missing_element(reference,liste):
  set1 = set(reference)
  set2 = set(liste)
  missing = list(sorted(set1 - set2))
  # print(missing)
  # pour extra elements
  # added = list(sorted(set2 - set1))
  # print(added)
  missing.sort()
  return missing
# Test de fonctionnement
# ref = ['Arg12b3f', 'Arg12b2f', 'Arg12b1f']
# detec  = ['Arg12b2f', 'Arg12b1f']
# missing = ['Arg12b3f']
# missing_element(ref,detec)


# Détection défaillance capteur bloqué à une valeur
# In: dataframe avec deux colonnes ('Id_Smart_Capteur + parametre')
# OUT: liste des valeurs retunues pour la moyenne, liste des capteurs bloquant, df de valeur à comparer
def recherche_defaillance(df,parametre,liste_SmS,verbose = False):
  # Extraction des mesures des trois capteurs de la liste pour rechercher des défaillance
  rdf = df.iloc[:0,:].copy()
  odf = df.iloc[:0,:].copy()
  if verbose:
    print("recherche défaillance")
    print(df)
    print(rdf)
    print(odf)
  values_param = []
  block = []
  for i in liste_SmS:
    # yp = df[df['Id_Smart_Capteur'] == i][parametre].to_list()
    # yp = df[df['Id_Smart_Capteur'] == i][parametre]
    yp = df[df['Id_Smart_Capteur'] == i]
    # on verifie avant si la liste ne contient pas qu'un seul element
    # sinon on va faire la moyenne d'une liste vide
    if verbose:
      print("taille du dataframe",len(yp.values))
      print(yp[parametre].values)


    if((len(yp)==1) | (len(yp)==2)):
      print("1 seul element dans la liste")
      values_param = values_param + yp[parametre].values.tolist()
      rdf = pd.concat([rdf,yp])

    else:
      # extract en une liste des valeurs du df
      if(len(set(yp[parametre].values))==1):
        if verbose:
          print("Eléments identiques pour",i)
          print(yp)
        block.append(i)
        odf = pd.concat([odf,yp])
      else:
        if verbose:
          print("Eléments différents pour",i)
          print(yp)
          print("Ajout des valeurs dans la liste values_param")
          print(yp[parametre].values, type(yp[parametre].values), values_param, type(values_param))
        values_param = values_param + yp[parametre].values.tolist()
        # print("liste des valeurs retunes")
        # print(values_param)
        rdf = pd.concat([rdf,yp])
        # print("reliable dataframe")
        # print(rdf)
  # print(block)
  block.sort()
  if verbose:
    print("=======fin========")
  return values_param , block , rdf, odf


# traitement du dataframe alertes pour avoir uniquement les changements
def pertinent_alert(alert,parametre):
  # Création d'un masque par ajout d'une colonne qui verifie si actuelle = suivante (True)
  m1 = (alert['Fault' + parametre] == alert['Fault' + parametre].shift(1))
  # Conserve les lignes qui sont False pour ne conserver que les changement d'état des capteurs
  return alert[~m1]

# Smoothing curve by apply kernel regression
# Input: time serie of reliable values
# Output time serie filtrated
def kernel_regression(X1):
  # Extract times series

  # print("Start filtering")
  # Must have two points minimum
  if(len(X1) == 0 or len(X1) == 1):
    X1f = X1
    # print("not enough data",X1)
  else:
    X1_ = X1.copy()
    X1_.index = linspace(1., len(X1_),len(X1_))
    kr = KernelReg([X1_.values],[X1_.index.values], var_type='u')
    f1 = kr.fit([X1_.index.values])
    X1f = pd.Series(data=f1[0], index = X1.index, dtype=float)
  # print("End filtering")
  # Return filtered data time-series
  return X1f

# Filtration: Use Kernel regression on parameter
# In: dataframe avec deux colonnes ('Id_Smart_Capteur + parametre')
# OUT: dataframe avec colonne supplémentaire ('parametref')
def filtration(df):
  # init variable de sortie
  filtered = df.iloc[:0,:].copy()
  # recupération de la liste des capteurs en service
  SmS_liste = df['Id_Smart_Capteur'].unique()
  # print("capteurs:",SmS_liste)
  # recupération du parametre à lisser
  col = df.columns
  # print("parametre:",col[1])
  parametre = col[1]
  # print("tranche à filter",df)
  for capteur in SmS_liste:
    X = df[df['Id_Smart_Capteur'] == capteur].copy()
    # print(capteur,X)
    X[parametre +'filtered'] = kernel_regression(X[parametre]).values
    # print("filtered",X)
    filtered = pd.concat([filtered,X])
    # print("tranche filtrée",filtered)
    filtered.sort_index(inplace=True)
  return filtered


# Fontion principale de synthèse

In [ ]:
# In: Dataframe, threshold seuil de tolérance, tranche de temps (Tau)
# In: parametre, liste des nom des capteurs de la station
# OUT: Synthesis of
def parametre_synthesis(df,threshold,tau,parametre,ref_SmS,verbose = False):
  # df(pd.dataframe,int,int,string,list of string)
  if verbose:
    print("polluant traité",parametre)
  # extraction df pour le parametre choisit
  df_parametre = df[['Id_Smart_Capteur', parametre]]
  # print(df_parametre)
  if verbose:
    print("taille du dataframe", len(df_parametre.index))
  # convertion int en string et récupération de la moitié pour décaler la tranche
  tau_demi = round(tau/2)
  tau = str(tau) + 'min'
  # tau_demi = str(tau_demi) + 'min'
  if verbose:
    print("Tranches de ",tau,"et décalage de ",tau_demi,"min")
  # 1 Decoupe le dataframe en tranches de temps
  t = tuple(df_parametre.resample(tau))
  if verbose:
    print("Nb de tranches:",len(t))
    print("StartTime df",t[0][0]," | EndTime df",t[-1][0]  )
  # print(t)
  # Création des listes pour dataframe de sortie
  sliceTime = []
  sliceTime_alert = []
  nb_sensor = []
  detect = []
  median = []
  median_voter = []
  indices = pd.DataFrame()
  outlier_slice = pd.DataFrame()
  reliable_slice = pd.DataFrame()
  filtered_slice = pd.DataFrame()
  reliable = df_parametre.iloc[:0,:].copy()
  outlier = df_parametre.iloc[:0,:].copy()
  # Pour chaque tranche de temps on traite les dataframes
  # Ici row fait référence à un timestamp row[0] + tranche df row[1]
  for row in t:

    # Nombre de capteur en service
    liste_SmS = row[1]['Id_Smart_Capteur'].unique()
    nb_sensor.append(len(liste_SmS))
    # Identification capteurs manquants
    Fault_SmS = {"missing" : missing_element(ref_SmS,liste_SmS)}
    # Ajout des capteurs manquants dans le dictionnaire des fautes
    detect.append(Fault_SmS)

    # Ajout de la date de l'alerte
    # sliceTime_alert.append(row[0])
    # décalage de la moitié

    sliceTime_alert.append(row[0] + timedelta(minutes = tau_demi) )

    # Si pas de capteur alors dataframe vide, on stop et on passe au suivant
    if row[1].empty:
      print("dataframe vide at ",row[0])
      continue


    # Recupération des mesures des capteurs du paramètre
    # Recherche et exclusion des mesures des capteurs bloqués à une valeur
    r = recherche_defaillance(row[1],parametre,liste_SmS)
    values_param = r[0]
    reprocess_values  = r[2]
    exclude_values = r[3]

    # Ajout dans le dictionnaire des fautes les capteurs concernés
    Fault_SmS['block'] = r[1]



    if verbose:
      print("Capteurs en service:",liste_SmS,"| Nb:",len(liste_SmS)," | Capteurs blockés:",Fault_SmS['block'],"| Nb:",len(Fault_SmS['block']),"\n")
      print("valeurs brutes")
      print(row[1],"\n")


    # Si l'ensembles des capteurs sont défaillants: arrêt du traitement
    if (len(Fault_SmS['block']) == len(liste_SmS)):
      print("Date:",row[0],"| L\'ensemble des capteurs sont en défaillance - mesures non utilisables")
      continue

    # print("time",row[0])
    # print("time+1/2",row[0] + timedelta(minutes = tau_demi))
    # Temps de la tranche (ajout de la demi tranche pour center)
    sliceTime.append(row[0] + timedelta(minutes = tau_demi))
    # sliceTime.append(row[0])

    # Calcul de la médiane des valeurs récoltées
    median_slice = calculate_median(values_param)
    median.append(median_slice)

    # Création seuils mobiles pour la detection des outliers defini avec threshold
    medmin = median_slice - (median_slice*threshold)/100
    medmax = median_slice + (median_slice*threshold)/100

    if verbose:


      print("exclusion")
      print(exclude_values,"\n")

      print("df à reprocess")
      print(reprocess_values,"\n")

    # Recherche des outliers dans les mesures des capteurs restants ]medmin,medmax[
    outlier_slice = reprocess_values[(reprocess_values[parametre] > medmax) | (reprocess_values[parametre] < medmin)]


    if verbose:
      print("outlier_slice")
      print(outlier_slice,"\n")

    # Recupération des mesures fiabilisée dans les seuils
    reliable_slice = reprocess_values[~reprocess_values.apply(tuple,1).isin(outlier_slice.apply(tuple,1))]

    if verbose:
      print("reliable slice")
      print(reliable_slice,"\n")

    # Fusion données exclues des capteurs défaillants avec outliers
    outlier_slice = pd.concat([outlier_slice,exclude_values])

    if verbose:
      print("New outlier_slice")
      print(outlier_slice,"\n")

      # Test: len row = len out + len rel
      print("time",row[0],"row[1]:", len(row[1].index) ," out:", len(outlier_slice.index), " rel:", len(reliable_slice.index) )

    # filtered slice
    # desctiver pour eviter la régression
    # filtered_slice = filtration(reliable_slice)
    # Activer cetteligne pour eviter le calcul de régression
    filtered_slice = reliable_slice
    # median voter
    voter_slice = calculate_median(filtered_slice[parametre].to_list())
    median_voter.append(voter_slice)
    # calcul de l'indice de fiabilité pour chaque capteur pour la tranche de temps Arg12b1f = local_df[local_df['Id_Smart_Capteur'] == 'Arg12b1f']
    indices_tranche = {}
    for sensor in liste_SmS:
      # extraction du nombre de points par capteur
      total_sensors_points = len( row[1][row[1]['Id_Smart_Capteur'] == sensor] )
      # extraction du nombre de points fiable
      reliable_sensor_points = len(reliable_slice[reliable_slice['Id_Smart_Capteur'] == sensor])
      # création de la ligne indice pour un capteur
      indice_line = { 'T' + sensor + parametre : total_sensors_points , 'R' + sensor + parametre : reliable_sensor_points}
      # mis à jour de la ligne du dictionnaire indices_tranches
      indices_tranche.update(indice_line)

    # Stockage valeurs dans dataframe global indices
    line = pd.DataFrame(indices_tranche,index=[row[0] + timedelta(minutes = tau_demi)])
    indices = pd.concat([indices,line])
    # indices = indices.append(line, ignore_index=False)
    # indices = indices.sort_index().reset_index(drop=True)


    # Store value in DataFrame
    outlier = pd.concat([outlier,outlier_slice])
    reliable = pd.concat([reliable,filtered_slice])


    if verbose:
      print("=====================================================")
      print("nbsensors:",len(nb_sensor),"|",nb_sensor,)
      print("detect:",len(detect),"|",detect)
      print("sliceTime_alert:",len(sliceTime_alert),"|",sliceTime_alert,)

  # Concatenation des resultats en dataframes
  synthese = pd.DataFrame(list(zip(median, median_voter)), index = sliceTime, columns=['median' + parametre, 'voteur' + parametre])

  alertes = pd.DataFrame(list(zip(nb_sensor, detect)), index = sliceTime_alert, columns=['Nb_' + parametre,'Fault' + parametre])


  detections = pertinent_alert(alertes,parametre)

  if verbose:
    print("Tailles des dataframes:","data:",len(df_parametre.index),"synthèse:",len(synthese.index),"ouliers:",len(outlier.index),"reliable:", len(reliable.index))
    print("Taille du df alertes:",len(alertes.index),"taille detections", len(detections))
    print("alertes")
    print(alertes)
    print("detections")
    print(detections)
    print("indices")
    print(indices)
  return synthese,outlier,reliable,detections,indices

# Fonction affichage pour S I D

In [ ]:

# Affiche les resultats de l'algorithme
def affiche_result(synthese,outlier,reliable,detections,threshold,tau,parametre,ref_SmS,IDx,Pc):
  # liste des défaillances
  print(detections)
  # Préparation des données
  y_upper = synthese['median' + parametre] + (threshold * synthese['median' + parametre])/100
  y_lower = synthese['median' + parametre] - (threshold * synthese['median' + parametre])/100
  # raw data
  X1out = outlier[outlier['Id_Smart_Capteur'] == ref_SmS[0]]
  X2out = outlier[outlier['Id_Smart_Capteur'] == ref_SmS[1]]
  X3out = outlier[outlier['Id_Smart_Capteur'] == ref_SmS[2]]
  E1 = len(X1out.index)
  # print(E1)

  X1rel = reliable[reliable['Id_Smart_Capteur'] == ref_SmS[0]]
  X2rel = reliable[reliable['Id_Smart_Capteur'] == ref_SmS[1]]
  X3rel = reliable[reliable['Id_Smart_Capteur'] == ref_SmS[2]]
  R1 = len(X1rel.index)

  # préparation données pour le calcul des indices
  # liste des indices
  # print(IDx.head(3))
  # Préparation des données
  Ih1 = IDx['R'+ref_SmS[0]+ parametre] / IDx['T'+ref_SmS[0]+ parametre]
  Ih2 = IDx['R'+ref_SmS[1]+ parametre] / IDx['T'+ref_SmS[1]+ parametre]
  Ih3 = IDx['R'+ref_SmS[2]+ parametre] / IDx['T'+ref_SmS[2]+ parametre]
  Id1 = 1 - Ih1
  Id2 = 1 - Ih2
  Id3 = 1 - Ih3
  # fonction heaviside pour dire que si supérieur à 2/3 alors c'est égal à 1
  Ic = np.heaviside(-Pc + Ih1.fillna(0),0) + np.heaviside(-Pc + Ih2.fillna(0),0) + np.heaviside(-Pc + Ih3.fillna(0),0)

  # Subplots with Shared X-Axes => row 1 = Synthesis; row 2= Indicateur de santé de la station; row 3=Detections par SmS
  fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.05, row_heights=[0.6, 0.2,0.2])




  # Titre de la figure
  fig.update_layout(height=650, width=950, title_text='S.I.D for ' + parametre +' (RƐ=' + str(threshold) + '%)' , title_x=0.2, title_y =0.98 ,title_font=dict(size=24), plot_bgcolor='rgba(0,0,0,0)')




  # Raw plots
  X1raw = pd.concat([X1out,X1rel])
  X2raw = pd.concat([X2out,X2rel])
  X3raw = pd.concat([X3out,X3rel])
  X1raw.sort_index(inplace=True)
  X2raw.sort_index(inplace=True)
  X3raw.sort_index(inplace=True)

  fig.add_trace(go.Scatter(
    x=X1raw.index,
    y=X1raw[parametre],
    mode='lines+markers',
    line=dict(width=2, color='blue'),
    marker=dict(size = 4, symbol = 'cross'),
    # legendgroup="S1",  # this can be any string, not just "S1"
    # legendgrouptitle_text="Synthesis data", #titre du groupe de legende
    name= u"\u03C6"+' SmS1',
  ),row = 1, col =1)

  fig.add_trace(go.Scatter(
    x=X2raw.index,
    y=X2raw[parametre],
    mode='lines+markers',
    line=dict(width=2, color='purple'),
    marker=dict(size = 4, symbol = 'circle'),
    # legendgroup="S1", # pour dire appartient au groupe de legende
    name= u"\u03C6"+' SmS2',
  ),row = 1, col =1)

  fig.add_trace(go.Scatter(
    x=X3raw.index,
    y=X3raw[parametre],
    mode='lines+markers',
    line=dict(width=2, color='darkgreen'),
    marker=dict(size = 4, symbol = 'x'),
    # legendgroup="S1",
    name= u"\u03C6"+' SmS3',
  ),row = 1, col =1)


  # # Ouliers plots
  # fig.add_trace(go.Scatter(
  #   x=X1out.index,
  #   y=X1out[parametre],
  #   mode="markers",
  #   name=ref_SmS[0] + 'out',
  #   marker = dict(size = 4, color = 'red', symbol = 'circle'),

  # ),row = 1, col =1)

  # fig.add_trace(go.Scatter(
  #   x=X2out.index,
  #   y=X2out[parametre],
  #   mode="markers",
  #   name=ref_SmS[1] + 'out',
  #   marker = dict(size = 4, color = 'red', symbol = 'x'),
  # ),row = 1, col =1)

  # fig.add_trace(go.Scatter(
  #   x=X3out.index,
  #   y=X3out[parametre],
  #   mode="markers",
  #   name=ref_SmS[2] + 'out',
  #   marker = dict(size = 4, color = 'red', symbol = 'cross'),
  # ),row = 1, col =1)

  # Affichages des seuils mobiles
  # fig.add_trace(go.Scatter(
  #   x=synthese.index,
  #   y=synthese['median' + parametre],
  #   line=dict(width=1, color='black'),
  #   name='Median' + parametre,
  # ),row = 1, col =1)

  fig.add_trace(go.Scatter(
    x=np.concatenate([synthese.index, synthese.index[::-1]]),
    y=pd.concat([y_upper, y_lower[::-1]]),
    fill='toself',
    line=dict(width=0.5, color='rgb(150, 247, 212)'),
    # fillcolor='rgb(184, 247, 212)',
    hoveron='points',
    # legendgroup="S1",
    name='Median bonds',
    # mode='none'
  ),row = 1, col =1)

  # Synthèse finale du paramètre
  fig.add_trace(go.Scatter(
    x=synthese.index,
    y=synthese['voteur' + parametre],
    line=dict(width=3, color='black'),
    # legendgroup="S1",
    # name='Synthesis ' + parametre,
    name='Synthesis '
  ),row = 1, col =1)

  # permet de rajouter la visualisation dynamique issue des stockcharts
  # fig.update_xaxes(
  #   rangeslider_visible=True,
  # )

  # Change grid color and axis colors
  # fig.update_xaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')
  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey',row = 1, col =1)

  # Titre de la figure
  # fig.update_layout(title_text='Synthese ' + parametre + ' (seuil:' + str(threshold) + '%)', title_x=0.5, plot_bgcolor='rgba(0,0,0,0)')

  # Indices de confiance et de défaillances
    # Création des figures indices row = 2


  fig.add_trace(go.Scatter(
    x=Id1.index,
    y=Id1,
    mode='lines+markers',
    line=dict(width=2, color='blue'),

    # legendgroup="Id",
    # legendgrouptitle_text="Indicators", #titre du groupe de legende
    line_shape='hv',
    name='Id SmS1',
  ),row = 3, col =1)

  fig.add_trace(go.Scatter(
    x=Id2.index,
    y=Id2,
    mode='lines+markers',
    line=dict(width=2, color='purple'),
    # legendgroup="Id",
    line_shape='hv',
    name='Id SmS2',
  ),row = 3, col =1)

  fig.add_trace(go.Scatter(
    x=Id3.index,
    y=Id3,
    mode='lines+markers',
    line=dict(width=2, color='darkgreen'),
    # legendgroup="Id",
    line_shape='hv',
    name='Id SmS3',
  ),row = 3, col =1)

  fig.add_trace(go.Scatter(
    x=Ic.index,
    y=Ic,
    mode='lines+markers',
    line=dict(width=3, color='orange'),
    # legendgroup="Id",
    line_shape='hv',
    name='Ic station',
  ),row = 2, col =1)




  # Annotation nombre d'erreurs des capteurs
  fig.add_annotation(dict(font=dict(color='black',size=16),
  x=1,
  y=1.2,
  showarrow=False,
  align="left",
  text=f"SmS1 : {len(X1out.index)} out. / {len(X1out.index) + len(X1rel.index)} obs. <br>SmS2 : {len(X2out.index)} out. / {len(X2out.index) + len(X2rel.index)} obs. <br>SmS3 : {len(X3out.index)} out. / {len(X3out.index) + len(X3rel.index)} obs.",
  textangle=0,
  xanchor='right',
  xref="paper",
  yref="paper"))


  # Ajout detections sur figure

  for index,  row in detections.iterrows():
    miss = row['Fault'+parametre].get('missing')
    freeze = row['Fault'+parametre].get('block')
    note = ""

    # if len(miss) != 0:
    #   # print("missing",miss)
    #   note += "miss:"
    #   note += str(miss)
    #   note += '\n'

    # if (freeze is not None):
    #   if (len(freeze) != 0):
    #     # print("freeze",freeze)
    #     note += "freeze:"
    #     note += str(freeze)

    # ajout de l'alerte sur le graphique
    # attention le timestamp est mal interprérté decalage d'une heure d'ou le -3600


    fig.add_vline(x= (index.timestamp()-3600) * 1000, line_width = 1, line_dash = 'dash', line_color = "red", annotation_text = note, annotation_position="top right",annotation_font_size = 8,annotation_textangle = 90,annotation_font_color="black",row = 1, col =1)


  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='black',zerolinecolor = 'black', title = "µg/m3", gridcolor='black',row = 1, col =1)
  # fig.update_yaxes(range=[0, 50], row=1, col=1)
  if parametre == 'Pression':
    fig.update_yaxes(showline=True, linewidth=0.5, linecolor='black', title = "hPa", gridcolor='black',row = 1, col =1)
  if parametre == 'Temperature':
    fig.update_yaxes(showline=True, linewidth=0.5, linecolor='black', title = "°C", gridcolor='black',row = 1, col =1)
  if parametre == 'Humidite':
    fig.update_yaxes(showline=True, linewidth=0.5, linecolor='black', title = "%Hr", gridcolor='black',row = 1, col =1)
    fig.update_layout(title_text='S.I.D for ' + 'Humidity' +' (RƐ=' + str(threshold) + '%)' )

  # Change grid color and axis colors
  # fig.update_xaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')

  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='grey',zerolinecolor = 'black', gridcolor='grey',row = 2, col =1)
  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='grey',zerolinecolor = 'black', gridcolor='grey',row = 3, col =1)

  fig.update_yaxes(range=[-0.1,1.1], row=3, col=1)
  # rectangle de validation
  fig.add_hrect(y0=0, y1=1-Pc, line_width=0.5, fillcolor="limegreen", opacity=0.35, row=3, col=1)
  # fig.update_layout(legend_tracegroupgap = 120)
  fig.update_layout(legend=dict(
      font=dict(
            size=16
        ),
    orientation="h",
    yanchor="bottom",
    y=-0.3,
    xanchor="center",
    x=0.5
  ))
  fig.show()

# Playground

## Création des différentes dataframes pour SID

In [ ]:

# data = dfin.loc['2022-01-24':'2022-05-22 9:00']
# data = dfin.loc['2022-02-07':'2022-02-15']
data = dfin.loc['2022-02-08':'2022-02-08']
# data = dfin.loc['2022-02-09':'2022-02-11']
# data = dfin.loc['2022-02-07 7:00':'2022-02-07 9:00']
# data = dfin.loc[date_debut : date_fin]
print(data[:1])
# nom de référence des SmS attribué à la station Arg
ref_SmS = ['Arg12b1f', 'Arg12b2f', 'Arg12b3f']
# Extraction des paramètres
parametres = data.columns.to_list()
# suppression de Id_SmS pour la liste des parametres
del parametres[parametres.index("Id_Smart_Capteur")]
print(parametres)# ['Humidite', 'Pression', 'PM10', 'PM25', 'Temperature']
# seuils de tolérance des variations des capteurs associés aux paramètres
seuils = [20,0.2,30,30,10]
# définition des tranches de temps
tau = 60
# list pour concatener dataframes
list_syntheses = []
list_detections = []
list_outliers = []
list_reliables = []
list_indices = []
verbose =  False
for n in range(len(parametres)):
  print(n,"seuil:",seuils[n],"% pour",parametres[n])
  threshold = seuils[n]
  parametre = parametres[n]
  print(parametre,threshold)
  result = parametre_synthesis(data,threshold,tau,parametre,ref_SmS,verbose)
  # storage dans les listes globales
  list_syntheses.append(result[0])
  list_outliers.append(result[1])
  list_reliables.append(result[2])
  list_detections.append(result[3])
  list_indices.append(result[4])
# Concatenations len out et reliable varient
global_outliers = pd.concat(list_outliers)
global_reliable = pd.concat(list_reliables)
# detection et synthèses ont la même longueur
global_detections = pd.concat(list_detections, axis = 1)
global_syntheses = pd.concat(list_syntheses, axis = 1)
global_indices = pd.concat(list_indices, axis = 1)

# print("=================synthèse=================")
# print(global_syntheses.head(2),"\n")
# print("================outliers==================")
# print(global_outliers.head(2),"\n")
# print("================reliable===================")
# print(global_reliable.head(2),"\n")
# print("================détections===================")
# print(global_detections.head(2),"\n")
print("="*10 , "FIN" , "="*10)

                    Id_Smart_Capteur  Humidite  Pression  PM10  PM2.5  \
time                                                                    
2022-02-08 00:03:30         Arg12b2f  43.10352  979.5614   8.4    4.3   

                     Temperature  
time                              
2022-02-08 00:03:30        18.08  
['Humidite', 'Pression', 'PM10', 'PM2.5', 'Temperature']
0 seuil: 20 % pour Humidite
Humidite 20
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seuil: 0.2 % pour Pression
Pression 0.2
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
2 seuil: 30 % pour PM10
PM10 30
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul element dans la liste
1 seul elem

# Sensors view

In [ ]:
# Affiche les mesures des capteurs, les suiles pour détéctions et les résultats des voteurs
# Permet d'aider à définir les seuils pour chaque type de capteur
# Dans un second temps support de décision pour maintenance
for n in range(len(parametres)):

  print(n,"seuil:",seuils[n],"% pour",parametres[n])
  threshold = seuils[n]
  parametre = parametres[n]
  print("Affichage des capteurs ",parametre," seuil choisit:",threshold)
  synthese = global_syntheses[['median'+ parametre,'voteur'+ parametre]].dropna()
  outlier = global_outliers[['Id_Smart_Capteur',parametre]].dropna()
  reliable = global_reliable[['Id_Smart_Capteur',parametre]].dropna()
  # detections = global_detections[['Nb','Fault'+ parametre]].dropna()
  detections = global_detections[['Fault'+ parametre]].dropna()
  IDx = global_indices.filter(regex=parametre)
  # Pc nombre de données juste sur le total
  Pc = 2/3
  affiche_result(synthese,outlier,reliable,detections,threshold,tau,parametre,ref_SmS,IDx,Pc )


0 seuil: 20 % pour Humidite
Affichage des capteurs  Humidite  seuil choisit: 20
                                              FaultHumidite
2022-02-08 00:30:00            {'missing': [], 'block': []}
2022-02-08 02:30:00  {'missing': ['Arg12b3f'], 'block': []}
2022-02-08 05:30:00            {'missing': [], 'block': []}
2022-02-08 07:30:00  {'missing': ['Arg12b3f'], 'block': []}
2022-02-08 08:30:00            {'missing': [], 'block': []}


1 seuil: 0.2 % pour Pression
Affichage des capteurs  Pression  seuil choisit: 0.2
                                              FaultPression
2022-02-08 00:30:00            {'missing': [], 'block': []}
2022-02-08 02:30:00  {'missing': ['Arg12b3f'], 'block': []}
2022-02-08 05:30:00            {'missing': [], 'block': []}
2022-02-08 07:30:00  {'missing': ['Arg12b3f'], 'block': []}
2022-02-08 08:30:00            {'missing': [], 'block': []}


2 seuil: 30 % pour PM10
Affichage des capteurs  PM10  seuil choisit: 30
                                                            FaultPM10
2022-02-08 00:30:00                      {'missing': [], 'block': []}
2022-02-08 02:30:00  {'missing': ['Arg12b3f'], 'block': ['Arg12b2f']}
2022-02-08 05:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 07:30:00  {'missing': ['Arg12b3f'], 'block': ['Arg12b2f']}
2022-02-08 08:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 09:30:00                      {'missing': [], 'block': []}
2022-02-08 19:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 20:30:00                      {'missing': [], 'block': []}
2022-02-08 22:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 23:30:00                      {'missing': [], 'block': []}


3 seuil: 30 % pour PM2.5
Affichage des capteurs  PM2.5  seuil choisit: 30
                                                           FaultPM2.5
2022-02-08 00:30:00                      {'missing': [], 'block': []}
2022-02-08 02:30:00  {'missing': ['Arg12b3f'], 'block': ['Arg12b2f']}
2022-02-08 05:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 07:30:00  {'missing': ['Arg12b3f'], 'block': ['Arg12b2f']}
2022-02-08 08:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 09:30:00                      {'missing': [], 'block': []}
2022-02-08 19:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 20:30:00                      {'missing': [], 'block': []}
2022-02-08 22:30:00            {'missing': [], 'block': ['Arg12b2f']}
2022-02-08 23:30:00                      {'missing': [], 'block': []}


4 seuil: 10 % pour Temperature
Affichage des capteurs  Temperature  seuil choisit: 10
                                           FaultTemperature
2022-02-08 00:30:00            {'missing': [], 'block': []}
2022-02-08 02:30:00  {'missing': ['Arg12b3f'], 'block': []}
2022-02-08 05:30:00            {'missing': [], 'block': []}
2022-02-08 07:30:00  {'missing': ['Arg12b3f'], 'block': []}
2022-02-08 08:30:00            {'missing': [], 'block': []}


# Synthese polluant + météorologie *locale*

In [ ]:
# Affiche les resultats de l'algorithme
def affiche_polluant_meteo(synthese_polluant,synthese_meteorologie,tau,polluant):
  # polluant
  print(synthese_polluant.head(2))
  # meteorologie
  print(synthese_meteorologie.head(2))

  # Préparation des données météorologiques
  y_Temperature = synthese_meteorologie['voteurTemperature']
  y_Humidite = synthese_meteorologie['voteurHumidite']
  y_Pression = synthese_meteorologie['voteurPression']
  x_meteorologie = synthese_meteorologie.index

  # Préparation des données polluant

  y_polluant = synthese_polluant['voteur' + polluant]
  x_polluant = synthese_polluant.index

  # Création des figures
  fig = go.Figure()


  # Synthèse finale du paramètre
  fig.add_trace(go.Scatter(
    x = x_polluant,
    y = y_polluant,
    line=dict(width=1.5, color='darkgreen'),
    name = polluant
  ))


  # Meteo plots

  fig.add_trace(go.Scatter(
    x = x_meteorologie,
    y = y_Humidite,
    line=dict(width=0.7, color='blue'),
    name='Humidité',
    yaxis="y2"
  ))

  fig.add_trace(go.Scatter(
    x = x_meteorologie,
    y = y_Temperature,
    line=dict(width=0.7, color='orange'),
    name='Temperature',
    yaxis="y3"
  ))

  fig.add_trace(go.Scatter(
    x = x_meteorologie,
    y = y_Pression ,
    line=dict(width=0.7, color='brown'),
    name='Pression',
    yaxis="y4"
  ))




  # permet de rajouter la visualisation dynamique issue des stockcharts
  # fig.update_xaxes(
  #   rangeslider_visible=True,
  # )

  # Change grid color and axis colors
  fig.update_xaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')
  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')

  # multi axes
  # Create axis objects
  fig.update_layout(
    xaxis=dict(
        domain=[0.1, 0.95]
    ),
    yaxis=dict(
        title=  "u/m3",
        titlefont=dict(
            color="darkgreen"
        ),
        tickfont=dict(
            color="darkgreen"
        )
    ),
    yaxis2=dict(
        title="Humidité",
        titlefont=dict(
            color="blue"
        ),
        tickfont=dict(
            color="blue"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0
    ),
    yaxis3=dict(
        title="Temperature °C",
        titlefont=dict(
            color="orange"
        ),
        tickfont=dict(
            color="orange"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="Pression hPa",
        titlefont=dict(
            color="brown"
        ),
        tickfont=dict(
            color="brown"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    )
  )
  # Titre de la figure
  fig.update_layout(title_text='Synthese ' + polluant + ' ', title_x=0.5, plot_bgcolor='rgba(0,0,0,0)')



  fig.show()

In [ ]:
# Affiche la synthèse d'un polluant en particulier avec les données météorolopgiques synthétisées
# permet de chercher des corrélation pour chaque type de polluant et météo

# selection des polluants parmis les paramètres
p_meteorologiques = ['Humidite', 'Pression', 'Temperature']
polluants  = missing_element(parametres,p_meteorologiques)
for n in range(len(polluants)):

  polluant = polluants[n]
  print("Affichage du polluant: ",polluant," avec météorologie locale")
  synthese_polluant = global_syntheses[['median'+ polluant,'voteur'+ polluant]].dropna()
  synthese_meteorologie = global_syntheses[['voteurHumidite','voteurPression','voteurTemperature']]

  affiche_polluant_meteo(synthese_polluant,synthese_meteorologie,tau,polluant)


Affichage du polluant:  PM10  avec météorologie locale
                     medianPM10  voteurPM10
2022-02-09 00:30:00         4.2        3.95
2022-02-09 01:30:00         2.5        2.40
                     voteurHumidite  voteurPression  voteurTemperature
2022-02-09 00:30:00        34.41504        975.4305              15.51
2022-02-09 01:30:00        33.69531        975.3492              15.39


Affichage du polluant:  PM2.5  avec météorologie locale
                     medianPM2.5  voteurPM2.5
2022-02-09 00:30:00          3.4          3.2
2022-02-09 01:30:00          1.8          1.8
                     voteurHumidite  voteurPression  voteurTemperature
2022-02-09 00:30:00        34.41504        975.4305              15.51
2022-02-09 01:30:00        33.69531        975.3492              15.39


# Import données extérieures


In [ ]:
import requests

url= 'https://github.com/spoupry/AQdata/raw/main/ARG_ATMO.xlsx'
myfile = requests.get(url)
atmo=pd.read_excel(myfile.content)
print(atmo.head(2))
atmo.dtypes
# Format et selection datas
# Déja converti en local datatime !
# select columns
data_select = atmo[["nom_com", "nom_polluant", "valeur", "date_debut"]]
# extraction for solve attribute value
Atmo_df = data_select.copy()
# Change name of columns without copy
Atmo_df.rename(columns = {'nom_com':'Lieu', 'nom_polluant':'Polluant', 'date_debut':'time'}, inplace = True)
# Setting the timestamp as index of the dataframe. THis helps to iterate through the data based on time
Atmo_df.set_index('time', inplace=True)
# Triage par index car c'est dans le désordre
Atmo_df.sort_index(inplace=True)
# Visualisation des polluants mesurés par la station ATMO
uniq_vals = Atmo_df['Polluant'].unique()
print("polluants mesurés:",uniq_vals)

          nom_com code_station nom_polluant  valeur   unite metrique  \
0  ARGELES-GAZOST      FR50813          NOX     6.3  ug.m-3  horaire   
1  ARGELES-GAZOST      FR50813          NO2     5.3  ug.m-3  horaire   

           date_debut            date_fin  
0 2022-04-19 05:00:00 2022-04-19 06:00:00  
1 2022-04-19 05:00:00 2022-04-19 06:00:00  
polluants mesurés: ['PM2.5' 'PM10' 'NO2' 'NOX' 'O3' 'NO']


Plage de temps identique à celle choisit avant

In [ ]:
Atmo_data = Atmo_df.loc[date_debut : date_fin]

In [ ]:
# # Découpage du dataframe par polluant pour visualisation
# PM25_Atmo = Atmo_data[Atmo_data["Polluant"] == "PM2.5"].valeur
# print(PM25_Atmo.head(3))
# PM10_Atmo = Atmo_data[Atmo_data["Polluant"] == "PM10"].valeur
# NO2_Atmo = Atmo_data[Atmo_data["Polluant"] == "NO2"].valeur
# NOX_Atmo = Atmo_data[Atmo_data["Polluant"] == "NOX"].valeur
# NO_Atmo = Atmo_data[Atmo_data["Polluant"] == "NO"].valeur
# O3_Atmo = Atmo_data[Atmo_data["Polluant"] == "O3"].valeur

# Affichage comparatif

In [ ]:
# Affiche les resultats de l'algorithme
def affiche_compare_ATMO_local(synthese_polluant,ATMO,polluant):
  # polluant ATMO
  polluant_Atmo = Atmo_data[Atmo_data["Polluant"] == polluant].valeur
  print(polluant_Atmo.head(3))
  # polluant
  print(synthese_polluant.head(2))

  # Préparation des données polluant

  y_polluant = synthese_polluant['voteur' + polluant]
  x_polluant = synthese_polluant.index

    # Préparation des données polluant

  y_polluant_Atmo = polluant_Atmo
  x_polluant_Atmo = polluant_Atmo.index

  # Création des figures
  fig = go.Figure()


  # Synthèse finale du paramètre
  fig.add_trace(go.Scatter(
    x = x_polluant,
    y = y_polluant,
    line=dict(width=1.5, color='darkgreen'),
    name = polluant
  ))

    # Affichage polluant ATMO
  fig.add_trace(go.Scatter(
    x = x_polluant_Atmo ,
    y = y_polluant_Atmo ,
    line=dict(width=1.5, color='darkblue'),
    name = "Atmo_" + polluant,
    yaxis="y2"
  ))

  # # permet de rajouter la visualisation dynamique issue des stockcharts
  # fig.update_xaxes(
  #   rangeslider_visible=True,
  # )

  # Change grid color and axis colors
  fig.update_xaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')
  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')

  # multi axes
  # Create axis objects
  fig.update_layout(
    xaxis=dict(
        domain=[0.1, 0.95]
    ),
    yaxis=dict(
        title=  "Local_u/m3",
        titlefont=dict(
            color="darkgreen"
        ),
        tickfont=dict(
            color="darkgreen"
        )
    ),
    yaxis2=dict(
        title= "Atmo_u/m3",
        titlefont=dict(
            color="blue"
        ),
        tickfont=dict(
            color="blue"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    )
  )
  # Titre de la figure
  fig.update_layout(title_text='Synthese ' + polluant + ' ', title_x=0.5, plot_bgcolor='rgba(0,0,0,0)')



  fig.show()

In [ ]:
# Affiche la synthèse d'un polluant en particulier avec les données météorolopgiques synthétisées
# Ajout de la comparaison avec la station ATMO la plus proche
# permet de chercher des corrélation pour chaque type de polluant et météo

# selection des polluants parmis les paramètres
p_meteorologiques = ['Humidite', 'Pression', 'Temperature']
polluants  = missing_element(parametres,p_meteorologiques)
for n in range(len(polluants)):

  polluant = polluants[n]
  print("Affichage du polluant: ",polluant," avec ATMO")
  synthese_polluant = global_syntheses[['median'+ polluant,'voteur'+ polluant]].dropna()


  affiche_compare_ATMO_local(synthese_polluant,Atmo_data,polluant)

Affichage du polluant:  PM10  avec ATMO
time
2022-02-08 00:00:00    35.5
2022-02-08 01:00:00    25.1
2022-02-08 02:00:00    17.0
Name: valeur, dtype: float64
                     medianPM10  voteurPM10
2022-02-09 00:30:00         4.2        3.95
2022-02-09 01:30:00         2.5        2.40


Affichage du polluant:  PM2.5  avec ATMO
time
2022-02-08 00:00:00    33.2
2022-02-08 01:00:00    23.6
2022-02-08 02:00:00    15.5
Name: valeur, dtype: float64
                     medianPM2.5  voteurPM2.5
2022-02-09 00:30:00          3.4          3.2
2022-02-09 01:30:00          1.8          1.8


In [ ]:
# Affiche les resultats de l'algorithme
def affiche_compare_ATMO_polluant_meteo(synthese_polluant,synthese_meteorologie,ATMO,tau,polluant):
  # polluant ATMO
  polluant_Atmo = Atmo_data[Atmo_data["Polluant"] == polluant].valeur
  print(polluant_Atmo.head(3))
  # polluant
  print(synthese_polluant.head(2))
  # meteorologie
  print(synthese_meteorologie.head(2))

  # Préparation des données météorologiques
  y_Temperature = synthese_meteorologie['voteurTemperature']
  y_Humidite = synthese_meteorologie['voteurHumidite']
  y_Pression = synthese_meteorologie['voteurPression']
  x_meteorologie = synthese_meteorologie.index

  # Préparation des données polluant

  y_polluant = synthese_polluant['voteur' + polluant]
  x_polluant = synthese_polluant.index

    # Préparation des données polluant

  y_polluant_Atmo = polluant_Atmo
  x_polluant_Atmo = polluant_Atmo.index

  # Création des figures
  fig = go.Figure()


  # Synthèse finale du paramètre
  fig.add_trace(go.Scatter(
    x = x_polluant,
    y = y_polluant,
    line=dict(width=1.5, color='darkgreen'),
    name = polluant
  ))

    # Affichage polluant ATMO
  fig.add_trace(go.Scatter(
    x = x_polluant_Atmo ,
    y = y_polluant_Atmo ,
    line=dict(width=1.5, color='darkblue'),
    name = "Atmo_" + polluant
  ))

  # Meteo plots

  fig.add_trace(go.Scatter(
    x = x_meteorologie,
    y = y_Humidite,
    line=dict(width=0.7, color='blue'),
    name='Humidité',
    yaxis="y2"
  ))

  fig.add_trace(go.Scatter(
    x = x_meteorologie,
    y = y_Temperature,
    line=dict(width=0.7, color='orange'),
    name='Temperature',
    yaxis="y3"
  ))

  fig.add_trace(go.Scatter(
    x = x_meteorologie,
    y = y_Pression ,
    line=dict(width=0.7, color='brown'),
    name='Pression',
    yaxis="y4"
  ))




  # permet de rajouter la visualisation dynamique issue des stockcharts
  # fig.update_xaxes(
  #   rangeslider_visible=True,
  # )

  # Change grid color and axis colors
  fig.update_xaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')
  fig.update_yaxes(showline=True, linewidth=0.5, linecolor='grey', gridcolor='grey')

  # multi axes
  # Create axis objects
  fig.update_layout(
    xaxis=dict(
        domain=[0.1, 0.95]
    ),
    yaxis=dict(
        title=  "u/m3",
        titlefont=dict(
            color="darkgreen"
        ),
        tickfont=dict(
            color="darkgreen"
        )
    ),
    yaxis2=dict(
        title="Humidité",
        titlefont=dict(
            color="blue"
        ),
        tickfont=dict(
            color="blue"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0
    ),
    yaxis3=dict(
        title="Temperature °C",
        titlefont=dict(
            color="orange"
        ),
        tickfont=dict(
            color="orange"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="Pression hPa",
        titlefont=dict(
            color="brown"
        ),
        tickfont=dict(
            color="brown"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=1
    )
  )
  # Titre de la figure
  fig.update_layout(title_text='Synthese ' + polluant + ' ', title_x=0.5, plot_bgcolor='rgba(0,0,0,0)')



  fig.show()

In [ ]:
# Affiche la synthèse d'un polluant en particulier avec les données météorolopgiques synthétisées
# Ajout de la comparaison avec la station ATMO la plus proche
# permet de chercher des corrélation pour chaque type de polluant et météo

# selection des polluants parmis les paramètres
p_meteorologiques = ['Humidite', 'Pression', 'Temperature']
polluants  = missing_element(parametres,p_meteorologiques)
for n in range(len(polluants)):

  polluant = polluants[n]
  print("Affichage du polluant: ",polluant," avec météorologie locale + comparaison avec ATMO")
  synthese_polluant = global_syntheses[['median'+ polluant,'voteur'+ polluant]].dropna()
  synthese_meteorologie = global_syntheses[['voteurHumidite','voteurPression','voteurTemperature']]

  affiche_compare_ATMO_polluant_meteo(synthese_polluant,synthese_meteorologie,Atmo_data,tau,polluant)

Affichage du polluant:  PM10  avec météorologie locale + comparaison avec ATMO
time
2022-02-08 00:00:00    35.5
2022-02-08 01:00:00    25.1
2022-02-08 02:00:00    17.0
Name: valeur, dtype: float64
                     medianPM10  voteurPM10
2022-02-09 00:30:00         4.2        3.95
2022-02-09 01:30:00         2.5        2.40
                     voteurHumidite  voteurPression  voteurTemperature
2022-02-09 00:30:00        34.41504        975.4305              15.51
2022-02-09 01:30:00        33.69531        975.3492              15.39


Affichage du polluant:  PM2.5  avec météorologie locale + comparaison avec ATMO
time
2022-02-08 00:00:00    33.2
2022-02-08 01:00:00    23.6
2022-02-08 02:00:00    15.5
Name: valeur, dtype: float64
                     medianPM2.5  voteurPM2.5
2022-02-09 00:30:00          3.4          3.2
2022-02-09 01:30:00          1.8          1.8
                     voteurHumidite  voteurPression  voteurTemperature
2022-02-09 00:30:00        34.41504        975.4305              15.51
2022-02-09 01:30:00        33.69531        975.3492              15.39


# Essais

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Arg12b1f.PM10.index,
    y=Arg12b1f.PM10,
    name='Arg12b1f',
))

fig.add_trace(go.Scatter(
    x=Arg12b2f.PM10.index,
    y=Arg12b2f.PM10,
    name='Arg12b2f',
))

fig.add_trace(go.Scatter(
    x=Arg12b3f.PM10.index,
    y=Arg12b3f.PM10,
    name='Arg12b3f',
))

# permet de rajouter la visualisation dynamique issue des stockcharts
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(title_text='PM10 measures', title_x=0.5)
fig.show()

NameError: ignored